In [0]:
cd ~/work/waroncars

In [0]:
plt = plotter()
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [0]:
import kalman

In [0]:
# kalman setup
std0 = np.array([5, 5, 1, 1])
k = kalman.KalmanTracker(4, std0)

In [0]:
# path params
N = 100
zϵ = np.array([5, 5, 1, 1])
z0 = np.array([100, 200, 100, 50])
z1 = np.array([600, 400, 110, 40])

In [0]:
# generate path
t_path = np.linspace(0, 1, N)
z_path = (1-t_path[:, None])*z0[None, :] + t_path[:, None]*z1[None, :]
z_obs = z_path + zϵ[None, :]*np.random.randn(N, 4)
plt.plot(t_path, z_obs);

In [0]:
def gen_kalman(kal, path):
    for i, obs in enumerate(path):
        kal.update(obs, restart=(i==0))
        yield kal.predict()

In [0]:
z_pred = np.vstack([x[:4] for x, P in gen_kalman(k, z_obs)])
plt.plot(t_path, z_pred-z_obs);

In [0]:
idx = 2
plt.plot(t_path, z_pred[:, idx], t_path, z_obs[:, idx]);